In [1]:
#from sklearn.cluster import KMeans
import numpy as np
import scipy as sp
import sys
sys.path.append('../msrv-xc/')
sys.path.append('../wdtools')

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from astropy.io import fits
from astropy.table import Table
from tqdm import tqdm

### Query
from astroquery.sdss import SDSS
from astroquery.gaia import Gaia
from astropy import constants as c

#import wdtools
import msrvxc
import pickle

In [2]:
with open('../msrv-xc/grids/interp_bosz.pkl', 'rb') as f:
    interp_bosz = pickle.load(f)
    
with open('../msrv-xc/grids/interp_bosz_norm.pkl', 'rb') as f:
    interp_bosz_norm = pickle.load(f)
    
wvl = np.load('../msrv-xc/grids/wavl.npy')

In [3]:
from astropy.table import Table, vstack

ms_sdss4_wd_sdss4 = Table.read('data/ms_sdss4_wd_sdss4.fits')
ms_gaia_wd_sdss4 = Table.read('data/ms_gaia_wd_sdss4.fits')
#ms_sdss5_wd_sdss4 = Table.read('sdss5/ms_sdss5_wd_sdss4.fits')
#
#ms_gaia_wd_sdss5 = Table.read('sdss5/ms_gaia_wd_sdss5.fits')
#ms_sdss5_wd_sdss5 = Table.read('sdss5/ms_sdss5_wd_sdss5.fits')

def strip(catalog):  
    drop_cols = []
    keep_cols = ['ms_source_id', 'ms_plate', 'ms_mjd', 'ms_fiberid', 'radial_velocity', 'radial_velocity_error']
        
    for key in catalog.keys():
        if key not in keep_cols:
            drop_cols.append(key)
            
    catalog.remove_columns(drop_cols)
    
    return catalog

ms_sdss4_wd_sdss4 = strip(ms_sdss4_wd_sdss4)
ms_gaia_wd_sdss4  = strip(ms_gaia_wd_sdss4 )
#ms_sdss5_wd_sdss4 = strip(ms_sdss5_wd_sdss4)
#ms_gaia_wd_sdss5  = strip(ms_gaia_wd_sdss5 )
#ms_sdss5_wd_sdss5 = strip(ms_sdss5_wd_sdss5)

catalog = vstack([ms_sdss4_wd_sdss4, ms_gaia_wd_sdss4])#, ms_sdss5_wd_sdss4, ms_gaia_wd_sdss5, ms_sdss5_wd_sdss5])

In [4]:
print(len(catalog[catalog['radial_velocity'].mask]))

19


In [5]:
catalog[['radial_velocity', 'radial_velocity_error']]

radial_velocity,radial_velocity_error
km / s,km / s
float32,float32
--,--
--,--
--,--
--,--
--,--
--,--
--,--
--,--


In [6]:
def continuum_normalize(wl, fl, ivar = None, avg_size = 150, ret_cont = False):
    
    fl_norm = np.zeros(np.size(fl))
    fl_cont = np.zeros(np.size(fl))
    
    ivar_yes = 0
    if ivar is not None:
        ivar_yes = 1
        ivar_norm = np.zeros(np.size(fl))
        
    for i in range(np.size(wl)):
        wl_clip = ((wl[i]-avg_size/2)<wl) * (wl<(wl[i]+avg_size/2))
        fl_cont[i] = np.median(fl[wl_clip])
        if ivar_yes:
            ivar_norm[i] = ivar[i]*np.median(fl[wl_clip])**2
    
    fl_norm = fl/fl_cont
    
    if ret_cont:
        if ivar_yes:
            return wl, fl_norm, ivar_norm, fl_cont
        else:
            return wl, fl_norm, fl_cont
    else:
        if ivar_yes:
            return wl, fl_norm, ivar_norm
        else:
            return wl, fl_norm

In [7]:
drops = []

for i in tqdm(range(len(catalog))):
    if catalog['radial_velocity'].mask[i]:
        #try:
        spec = SDSS.get_spectra(plate=catalog['ms_plate'][i], fiberID=catalog['ms_fiberid'][i], mjd=catalog['ms_mjd'][i])[0]
    
        wl = (10**spec[1].data['loglam'])
        fl = (spec[1].data['flux'])
        ivar = (spec[1].data['ivar'])
        
        wl, fl, ivar = continuum_normalize(wl, fl, ivar)
        
        mask = (4500 < wl)*(wl < 5500)
        
        res, rv, e_rv, rvgrid, cc = msrvxc.fit.fit_rv(interp_bosz_norm, wvl, wl, fl, ivar, plot = False)
        
        catalog['radial_velocity'][i] = rv
        catalog['radial_velocity_error'][i] = e_rv
        #except:
        #    print('err')
        #    drops.append(i)
            
catalog.remove_rows(drops)

  2%|█▉                                                                                 | 4/173 [00:40<28:44, 10.20s/it]/home/arseneau/anaconda3/lib/python3.9/site-packages/astroquery/sdss/core.py:874: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  arr = np.atleast_1d(np.genfromtxt(io.BytesIO(response.content),
100%|█████████████████████████████████████████████████████████████████████████████████| 173/173 [03:34<00:00,  1.24s/it]


In [8]:
catalog[['radial_velocity', 'radial_velocity_error']]

radial_velocity,radial_velocity_error
km / s,km / s
float32,float32
200.39432,3.4003677
87.07357,7.071068
51.984837,14.783859
101.99453,30.766256
80.701256,7.566373
-2.6788187,2.6925824
25.877562,5.0990195
27.10566,10.547511


In [9]:
catalog.write('data/04_ms_rv.fits', overwrite = True)